## Scan for Yaml files:

In [ ]:
from pathlib import Path

benefits_folder = Path('./aides')

def extract_benefits_paths(benefits_folder):
    isYAMLfile = lambda path : str(path).endswith('.yml') or str(path).endswith('.yaml')
    liste_fichiers = [str(benefit) for benefit in benefits_folder.iterdir() if isYAMLfile(benefit)]
    return liste_fichiers

benefits_files_paths = extract_benefits_paths(benefits_folder)
benefits_files_paths

## Open and parse files: make a list of Dict

In [ ]:
import yaml

def extract_benefit_file_content(aide_path):
    return yaml.safe_load(open(aide_path))

def extract_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [extract_benefit_file_content(path) for path in benefits_files_paths]
    return benefits

In [ ]:
from benefit import Benefit
def parse_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [Benefit(extract_benefit_file_content(path)) for path in benefits_files_paths]
    return benefits

benefits_objects = parse_benefits_from_yamls(benefits_folder)    

In [ ]:
benefits = extract_benefits_from_yamls(benefits_folder)

##  Make a list of [profils] [type] in all aides given

In [ ]:
from functools import reduce
from benefit import BenefitProfile

def accumulate_profils(profils: list, benefit: BenefitProfile):
        return profils + benefit.get_profils_types()

profiles_names = reduce(accumulate_profils, benefits_objects, [])


# Make a list with all conditons:
	- conditions generales
	- conditions in [profils][type]

In [ ]:
from functools import reduce


def extract_conditions(benefit):
    
    conditions = [condition['type'] for condition in benefit['conditions_generales']]
    profil_conditions = [condition['type'] for profil in benefit['profils'] if 'conditions' in profil for condition in profil['conditions'] ]
    
    return conditions + profil_conditions

def accumulate_conditions(conditions: list, benefit: dict):
    return conditions + extract_conditions(benefit)


conditions_names = reduce(accumulate_conditions, benefits, [])
conditions_names

# Count frequence of profils and conditions appearances

In [ ]:
def count_list_item_frequence(list : list):
    elements_counter = {}
    for element in list:
        if element in elements_counter:
            elements_counter[element] +=1
        else:
            elements_counter[element] = 1
    return (sorted(elements_counter.items(), key=lambda x:x[1]))

profil_names_frequence = count_list_item_frequence(profiles_names)
condition_frequence = count_list_item_frequence(conditions_names)

In [ ]:
condition_frequence

In [ ]:
profil_names_frequence

## Reference benefits with no profils:

In [ ]:
def contains_profils(benefit :dict):
    return 'profils' in benefit and benefit['profils'] != []

def extract_benefit_without_profile(benefit):
    if contains_profils(benefit):
        return []
    else:
        return [f"{benefit['label']} => {benefit['institution']}"]

def accumulate_benefit_without_profile(list:list, benefit):
    
    return list + extract_benefit_without_profile(benefit)
    
    
benefits_without_profils = reduce(accumulate_benefit_without_profile, benefits, [])
benefits_without_profils

## Benefits using only age as condition:

In [ ]:
def is_only_condition_age(benefit):
    conditions = extract_conditions(benefit)
    return all(condition == "age" for condition in conditions)

def extract_benefit_with_only_condition(benefit):
    if is_only_condition_age(benefit):
        return [f"{benefit['label']} => {benefit['institution']}"]
    else:
        return []

def accumulate_benefit_with_only_condition(list:list, benefit):
    
    return list + extract_benefit_with_only_condition(benefit)

benefits_with_only_age_condition = reduce(accumulate_benefit_with_only_condition, benefits, [])
benefits_with_only_age_condition